In [1]:
# 라이브러리 설치
!pip install feedparser
!pip install newspaper3k
!pip install konlpy
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=436404fcd0f7ba4fbb329ffc9d051054753458d8d77f2e1def477cd703b1d07c
  Stored in directory: /root/.cache/pip/wheels/65/7a/a7/78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 1.3 MB/s eta 0:00:00
  Created wheel for tinyseg

In [2]:
# 라이브러리 불러오기
import feedparser # feed를 파싱하는 모듈
from newspaper import Article # 신문기사를 불러오는 모듈
from konlpy.tag import Okt # 형태소 분석 모듈
from bs4 import BeautifulSoup # HTML을 파싱하는 모듈
import numpy as np # 넘파이는 다 아는 라이브러리죠?
from collections import Counter # iterbale 객체에서 각 요소의 등장 횟수를 세는 모듈
from operator import eq # '=='와 동일한 역할을 하는 비교연산자 모듈

In [3]:
# 상위 4개 링크 'urls '리스트에 저장
urls = ["http://rss.etnews.com/Section901.xml",
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml",
        "http://rss.etnews.com/Section904.xml"]

In [4]:
# [단계#1] 제목과 링크를 크롤링하는 함수 선언
def crawl_rss(urls):
  array_rss = [] # 빈 리스트 생성
  for url in urls:
    # print('[Crawl RSS] ', url) # 진행 과정을 보여준다 (없어도 되는과정)
    parse_rss = feedparser.parse(url) # 파싱
    for p in parse_rss.entries: # 아래에서 title, link 외에 다른 정보들도 가져올 수 있음 (파싱 전 태그들을 확인하고 사용)
      array_rss.append({'title':p.title, 'link':p.link}) # 타이틀과 링크를 리스트에 추가
  return array_rss # 완성된 리스트 반환

In [5]:
# 기사들의 제목과 link를 크롤링 해보자 !
list_articles = crawl_rss(urls)

In [6]:
# 크롤링 한 것이 제대로 저장 되었나 확인해보기 (첫번째 기사만)
print(f'''타이틀 : {list_articles[0]['title']}
링크 : {list_articles[0]['link']}''') # 첫번째 기사의 title과 link가 출력

타이틀 : 권도형 누가 데려갈까...몬테네그로 "美, 韓보다 먼저 인도 청구"
링크 : https://www.etnews.com/20230330000082


In [7]:
# [단계#2] 본문 크롤링 함수 만들기
def crawl_article(url, language='ko'):
  # print('[Crawl Article] ', url) # 진행 과정을 보여준다 (없어도 되는과정)
  temp = Article(url, language = language) # Article은 본문을 긁어오는 함수이다
  temp.download() # 기사 본문을 다운로드
  temp.parse() # 기사 본분을 파싱
  return temp.title, temp.text # 기사의 제목과 내용(텍스트)를 반환

In [8]:
# 본문을 크롤링 해보자 !
for article in list_articles:
  _, text = crawl_article(article['link']) # 앞의 '_'부분은 title이 들어가는 부분이라, 생략의 의미에서 _를 사용
  article['text'] = text

In [9]:
# 가져온 본문을 확인 (첫번째 기사만 보기)
print(f'''<<아래는 기사 본문 내용>>>

{list_articles[0]['text']}''') # 첫번째 기사의 본문을 출력

<<아래는 기사 본문 내용>>>

몬테네그로 당국은 29일(현지시간) 자국에서 체포돼 구금 중인 가상화폐 '테라·루나' 사태의 핵심 인물인 권도형(32) 테라폼랩스 대표에 대해 지금까지 한국과 미국, 두 나라가 범죄인 인도 청구를 했다고 확인했으며 "미국이 한국보다 먼저 범죄인 인도 청구를 했다"고 밝혔다.



마르코 코바치 법무부 장관은 이날 몬테네그로 수도 포드고리차의 프레스센터에서 기자회견을 열고 이같이 전했다.



주세르비아 한국 대사관은 전날 몬테네그로 외교부·법무부 관계자들과 만나 권 대표의 조속한 송환에 대한 협조를 요청했다. 몬테네그로에는 우리 대사관이 설치돼 있지 않아 인접 국가인 세르비아 대사관이 몬테네그로를 관할하고 있다.



한국과 달리 몬테네그로에 대사관을 둔 미국 정부는 이미 확보한 외교 채널을 가동, 권 대표에 대해 우리보다 먼저 범죄인 인도 청구를 한 것으로 확인됐다.



코바치 장관은 권 대표가 어느 국가로 송환될지는 범죄의 중요성, 범죄인 국적, 범죄인 인도 청구 날짜를 기준으로 결정한다고 설명했다. 국제법상 피의자를 체포한 국가가 송환국을 정한다.



미국이 범죄인 인도 청구 날짜에서 앞섰다는 점은 한국과의 '송환 경쟁'에서 좀 더 유리한 요소가 될 수 있다. 다만 송환 국가를 정할 때는 범죄인 국적도 큰 영향을 미치기에 미국과 한국 중 어느 쪽에 우선권이 갈지는 아직 예단하기 어렵다는 관측이 나온다.



코바치 장관 역시 "현 단계에서 두 국가 중 어느 쪽이 우선권이 있는지 말하기 어렵다"며 "싱가포르도 아직 공식적으로 범죄인 인도 요청을 하지 않았지만 우리는 싱가포르에서 형사 소송이 진행되고 있다는 것을 알고 있다"고 말했다.



권 대표는 측근인 한모 씨와 함께 지난 23일 포드고리차 국제공항에서 코스타리카 위조 여권을 갖고 아랍에미리트(UAE) 두바이행 비행기에 탑승하려다 검거됐다.



권 대표는 가상화폐 테라와 루나 폭락 사태 직전인 지난해 4월 싱가포르로 출국한 뒤 잠적했다. 이후 두바이를 거쳐 세르비

In [10]:
# get_keywords 함수 생성
def get_keywords(text, nKeywords = 10): # nKeywords의 디폴트를 10으로 줌
  spliter = Okt() # 형태소 분석기를 호출
  nouns = spliter.nouns(text) # 형태소 분석기를 통해 명사를 추출
  count = Counter(nouns) # 각각의 명사가 몇번이 나오는지 카운팅
  list_keywords = [] # 키워드를 담을 빈 리스트 생성
  for n, c in count.most_common(nKeywords): # .most_common : 카운트 수가 제일 높은 것 부터 가져옴
    item = {'keyword': n, 'count': c} # item 딕셔너리에 키워드와 나온횟수(카운팅)을 넣어주기
    list_keywords.append(item)
  return list_keywords

# [단계#3] 모든 본문 text에서 명사만 추출하기
for article in list_articles:
  keywords = get_keywords(article['text'])
  article['keywords'] = keywords

In [11]:
# 가져온 키워드를 확인 (첫번째 기사만 보기)
print(list_articles[0]['keywords']) # 첫번째 기사의 키워드들을 출력

[{'keyword': '몬테네그로', 'count': 10}, {'keyword': '대표', 'count': 9}, {'keyword': '범죄인', 'count': 8}, {'keyword': '인도', 'count': 8}, {'keyword': '권', 'count': 8}, {'keyword': '국가', 'count': 7}, {'keyword': '한국', 'count': 6}, {'keyword': '미국', 'count': 5}, {'keyword': '청구', 'count': 5}, {'keyword': '송환', 'count': 5}]


In [12]:
#단계 4 검색어를 입력받아서 그 검색어를 가지고있는 기사 출력
query = input()
def search_articles(query,keyword):
  nWords = 0 #아래 if문에 걸리지 않으면 (즉, 쿼리가 키워드에 없음)
  for kw in list_keywords: #키워드 리스트 검색
    noun = kw['keyword']
    count = kw['count']
    if eq(query, noun): # 쿼리와 동일한 키워드가 존재한다면
      nWords = count # nwords는 그 횟수(count)로 갱신
  return nWords

for article in list_articles:
  nQuery = search_articles(query,article['keywords'])
  if nQuery !=0:
    print('[TF]:',nQuery, '[Title]: ',article['title'], '[URL]:' ,article['link'])

KeyboardInterrupt: ignored